In [114]:
import openai,time,pandas,configur
from openai import OpenAI
from statistics import mean,median
import pandas as pd
import inspect

In [115]:
def ask_chatgpt(model,system_content,prompts):
    try:
        responses = []
        client = OpenAI(api_key = configur.thepass['gpt_api'])
        system_role = {"role": "system", "content": system_content}
        if model == 'gpt-3.5-turbo':
            for idx_prompt in prompts:
                index, prompt = idx_prompt[0],idx_prompt[1]
                completion = client.chat.completions.create(model=model,
                            messages=[system_role, {"role": "user", "content": prompt}])
            # Extracts the text from the response object
                answer_text = completion.choices[0].message.content
                responses.append(f'{str(index)} {answer_text}') 
            return '\n'.join(responses)
        elif model == 'gpt-4-turbo':
            for msg_gpt in prompts:
                msg = msg_gpt[0]
                gpt_response = msg_gpt[1]
                prompt = msg +'.. GPT Response: ' +gpt_response
                completion = client.chat.completions.create(model=model,
                            messages=[system_role, {"role": "user", "content": prompt}])
                answer_text = completion.choices[0].message.content
                responses.append(f'{msg_gpt} {answer_text}') 
            return '\n'.join(responses)
    
    except openai.APIError as e:
        print(f"API Error: {e}")
        time.sleep(60)
        return ask_chatgpt(prompt)
    except openai.APIConnectionError as e:
        print(f"Connection Error: {e}")
        time.sleep(60)
        return None
    except openai.RateLimitError as e:
        print(f"Rate Limit Error: {e}")
        time.sleep(60)
        return ask_chatgpt(prompt)

In [116]:
def read_msg_df(csv_file):
    msg_df = pandas.read_csv(f'{csv_file}',encoding='utf-8',index_col = 0)
    msg_df.head(10)
    return msg_df

In [117]:
def process_responses(response_text, response_dict):
    split_responses = response_text.split('\n')
    for i,response in enumerate(split_responses):
        if response.strip():
            parts = response.split(maxsplit=1)
            if len(parts) >= 2:
                try:
                    index = int(parts[0])  
                    response_text = parts[1] if len(parts) > 1 else "No response from GPT"
                    response_dict[index] = response_text
                except Exception as e:
                    print(e)

In [118]:
def send_split_data(path,turbo35_prompt,debugging=False):
    df = read_msg_df(path)
    responses = []
    accumulated_messages = []
    sum_so_far = 0
    response_dict = {}
    if debugging:
        df = df.tail(50)
    # Goes through the msgs and sends optimal batch length for our token limit
    for index, row in df.iterrows():
        msg = row['Message']
        word_count = len(msg.split())
        sum_so_far += word_count
        accumulated_messages.append((index,msg))
        
        if sum_so_far > 1300:
            response_text = ask_chatgpt("gpt-3.5-turbo", turbo35_prompt, accumulated_messages)
            print(f"Response before splitting: {response_text}")  # Debug output
            process_responses(response_text, response_dict)
            accumulated_messages = []
            sum_so_far = 0
    # Handles the last batch
    if accumulated_messages:
        response_text = ask_chatgpt("gpt-3.5-turbo", turbo35_prompt, accumulated_messages)
        process_responses(response_text, response_dict)
    try:
        response_df = pd.DataFrame.from_dict(response_dict, orient='index', columns=['ChatGPT_Response'])
        df = df.join(response_df, how='left')
        df['ChatGPT_Response'].fillna('Neither', inplace=True)
    except ValueError as e:
        print(f"DataFrame creation error: {e}")
        return None
    return df

In [119]:
def df_to_csv(df,path):
    df.to_csv(path)

In [124]:
def buy_or_sell(message):
    return message.split()[0]

In [131]:
def return_stock(message):
    for substring in ['!', '.', ':', ',', '\r', '\n', '*','_','$','[',']','None','0DTE','DD']:
        message = message.replace(substring, '')
    if len(message.split())>1:
        message = message.split()[1]
    if message.strip() in ['Buy','-','Ticker'] or len(message.strip()) > 5:
        return ''
    return message

In [125]:
def return_stock_df(gpt_df):
    
    gpt_df['Stock'] = gpt_df['ChatGPT_Response'].apply(lambda x: return_stock(x))
    gpt_df['Buy or Sell'] = gpt_df['ChatGPT_Response'].apply(lambda x: buy_or_sell(x))

    stock_df = gpt_df[(gpt_df['Stock']!='') & (gpt_df['Buy or Sell'] != 'Neither')]
    return stock_df

In [129]:
def debugging_gpt(df,path,old_prompt):
    extra_note = "Current big issue this prompt fails on: X TO Y if X < Y should be SELL since this is in profit. We SELL WHENEVER THE PLAY IS IN PROFIT"
    example = "appl lottos almost 100 percent from 09 for 195... should be Sell AAPL"
    gpt4_system = f'We are prompt engineering GPT-3.5-turbo, please help revise a better prompt (if the answer was wrong) that will improve accuracy. {extra_note}. {Example}. Return a full list of wrong ids. Old prompt: """{old_prompt}""""""'
    tuples = []
    for msg,gpt_response in df.itertuples(index=False):
        tuples.append((msg,gpt_response))
    response_text = ask_chatgpt("gpt-4-turbo", gpt4_system, tuples)
    print(response_text)


In [121]:
def main():
    debugging = True
    gpt_df = pd.DataFrame()
    turbo35_prompt = "System: Analyze the following trading messages. For each message, determine if the content suggests a [Buy/Sell/Neither] action for each [Ticker]. Consider 'added' and 'ready' as indicators for a buy signal; 'up' and 'reverses' as indicators for a sell signal. If no ticker is specified, use 'No Ticker Found'. Respond in the format '[Action] [Ticker]', explain your reasoning if the language is ambiguous."
    big_prompt = "System: Analyze the following trading messages. For each message, determine if the content suggests a [Buy/Sell/Neither] action for each [Ticker]. Consider 'added' and 'ready' as indicators for a buy signal; 'up' and 'reverses' as indicators for a sell signal. If no ticker is specified, use 'No Ticker Found'. Respond in the format '[Action] [Ticker]', explain your reasoning if the language is ambiguous."
    old_prompts = ["Summarize these messages into [Buy/Sell/Neither] [Ticker]. Use 'ready', 'added' as buy signals; 'up', 'reverses' as sell signals.",
        "Summarize these messages (<10 words) into [Buy/Sell/Neither] [Ticker] based on keywords. 'added' implies Buy, 'up' implies Sell."]
    disc_path = r'C:\Users\amoog\Desktop\Project_X\Project_X\disc_msg_df.csv'
    gpt_path = r"C:\Users\amoog\Desktop\Project_X\Project_X\gpt_msg_df.csv"
    if debugging:
#         gpt_df = send_split_data(disc_path,turbo35_prompt,True)
        stock_df = return_stock_df(gpt_df)
        debugging_gpt(df[['Message','ChatGPT_Response']],r"C:\Users\amoog\Desktop\Project_X\Project_X\gpt_msg_df.csv",turbo35_prompt)
    
        return gpt_df
    else:
        gpt_df = send_split_data(disc_path,turbo35_prompt,False)
        stock_df = return_stock_df(gpt_df)
#         df_to_csv(gpt_df,gpt_path)
        return stock_df

df = main()


Response before splitting: 0 Neither No Ticker Found. The message does not provide a clear indication to initiate a buy or sell action for any specific ticker.
1 No Ticker Found
2 No Ticker Found
3 No Ticker Found
4 No Ticker Found
5 Neither MSFT

This message provides historical context about the resistance level for MSFT at 290. It does not give a clear indication of whether to buy or sell the stock.
6 Neither No Ticker Found. This message does not provide a clear indication to buy or sell any specific stock. It seems to be a general comment on the trading environment with stops set at 200.
7 Sell LCID. The message indicates that LCID is up 20 percent, which is typically a signal to consider selling.
8 Neither No Ticker Found

This message does not provide a clear indication of a Buy or Sell action for any specific ticker. It mentions positive earnings and holding up strong, but does not give a specific direction.
9 Neither SNOW

This message does not contain a clear indication to bu

Response before splitting: 136 No Ticker Found.
137 Buy NFLX, Buy SPY

- NFLX: The message implies that there are winning trades for NFLX, so it is a positive signal to potentially buy.
- SPY: Similar to NFLX, the message indicates that there is a winning trade for SPY, suggesting a buy action.

No specific signal for SHOP was mentioned in the message, so no action is recommended for SHOP.
138 Buy SPX
139 Sell SPX

The message suggests selling SPX as the price has reached a significant level of 460, indicating a bearish sentiment.
140 Sell SPX

The message suggests a reduction in risk for the SPX, indicating a possible sell signal.
141 No Ticker Found.
142 Neither SPX
143 No Ticker Found
144 Neither SPX

Explanation: The message does not provide a clear signal to buy or sell. It only mentions the current price range of 340-560 for SPX and indicates that the sender is holding 1/2 of their position.
145 Sell WOWW. The message indicates a significant increase in the price of WOWW, suggest

Response before splitting: 271 Sell SPX
- The language "Stopped out" and "euphoric" suggest a negative sentiment, signaling a sell action for SPX.
272 Neither No Ticker Found

This message does not contain any specific information about stocks or trading actions, so no action is suggested.
273 Neither No Ticker Found

- The message does not clearly suggest a specific action to be taken for any ticker. The recommendation to "stay light" implies cautiousness rather than a specific buy or sell action.
274 No Ticker Found
275 Buy NVDA
- The mention of more NVDA and holding the majority suggests a positive sentiment towards the stock, indicating a buy action for NVDA.
276 No Ticker Found
277 Sell SPX
This message suggests selling SPX as the price has risen from 280 to 360 and the sender is "Holding most."
278 Buy No Ticker Found
279 Neither No Ticker Found. The message is discussing options trading ranges for the SPY without a clear indication of a buy or sell action.
280 Neither No Ticker 

Response before splitting: 396 Neither SPX

This message does not suggest a clear buy or sell action for the S&P 500 index. The statement indicates caution due to potentially inflated implied volatility (IV) and a plan to wait until after the Federal Open Market Committee (FOMC) meeting before making any decisions.
397 No Ticker Found
398 Sell Tesla. The language "600 percent almost" suggests a significant increase in Tesla's price, typically indicating a potential overvaluation and a sell opportunity.
399 Neither No Ticker Found

This message does not contain any specific trading instructions or signals.
400 Buy NFLX. The user added to their position in NFLX which suggests confidence in the stock's performance.
401 Buy TSLA

This message suggests a positive outcome for Tesla (TSLA) with a remark about a significant increase in value. The tone indicates a positive sentiment, which could prompt a buy action.
402 Neither NVDA
- The message does not provide a clear indication to buy or se

Response before splitting: 526 Sell No Ticker Found

The message suggests selling options, specifically puts, for maximum gain at a price of 410.
527 No Ticker Found
528 Buy No Ticker Found

- The message suggests a plan to enter a position between $31 to $34, which indicates a buying intention.
529 Neither No Ticker Found

This message does not provide a clear indication of a buy or sell action for any specific ticker.
530 Neither SPX
531 No Ticker Found
532 No Ticker Found
533 Neither No Ticker Found
534 No Ticker Found.
535 Sell NVDA. The message indicates that the trail has stopped and implies a potential downtrend based on the phrase "holding 422".
536 Sell No Ticker Found. The phrase "does not want this under 4520" suggests a negative outlook on the market in general, indicating a potential sell action.
537 Neither No Ticker Found
538 No Ticker Found
539 Buy DWAC. The message suggests a potential significant increase in price based on the pattern and news mentioned.
540 No Ticker

...

Response before splitting: 5266 No Ticker Found
5267 Sell SPX
- The message suggests trimming the position for SPX, indicating a sell action.
5268 No Ticker Found
5269 Neither AAPL

Explanation: The message does not provide a clear buy or sell signal for AAPL. It only suggests conducting due diligence and making a decision based on individual research. Therefore, no specific action (buy or sell) is recommended for AAPL in this message.
5270 Neither NFLX
5271 Sell SAVA

This message suggests a sell action for ticker SAVA, as it mentions that the stock price broke down below $25.
5272 Buy DWAC. The message indicates a positive sentiment towards DWAC with "nice".
5273 Neither SPY. The message does not provide a clear indication to buy or sell SPY.
5274 Buy AMC. The message suggests a bullish sentiment by mentioning AMC along with a call option and a specific price point.
5275 Neither No Ticker Found

- The message does not provide specific information or signals regarding a particular sto

Response before splitting: 5391 No Ticker Found
5392 Sell NVDA. The language "broke" and "flush" suggest a negative sentiment, indicating a sell action for NVDA.
5393 No Ticker Found.
5394 Neither - No Ticker Found

This message does not provide enough information to suggest a specific action for a particular ticker.
5395 Neither No Ticker Found
5396 Neither No Ticker Found

The message does not provide a clear indication to either buy or sell any specific ticker. It suggests that there are no planned plays for the night.
5397 No Ticker Found
5398 Neither No Ticker Found

Explanation: The message does not provide clear indication to prompt a buy or sell action for any specific ticker. The mention of staying light suggests a cautious approach, which does not convey a strong buy or sell signal.
5399 Neither No Ticker Found. The message does not provide a clear indication to buy or sell any specific stock.
5400 Neither NVDA

The message does not provide a clear signal to buy or sell NVDA.

Response before splitting: 5538 [Neither] NFLX

The message does not provide a clear buy or sell signal for NFLX. It encourages conducting due diligence to make an investment decision without indicating a specific direction.
5539 No Ticker Found
5540 No Ticker Found
5541 Neither SPX
5542 Neither No Ticker Found. This message does not provide a clear indication to buy or sell any specific asset.
5543 No Ticker Found
5544 No Ticker Found
5545 Neither COST
5546 No Ticker Found
5547 No Ticker Found
5548 Neither No Ticker Found

This message does not provide a clear indication to take any specific action in relation to a particular ticker.
5549 Neither SPX. The message does not clearly indicate a buy or sell action for SPX.
5550 Neither No Ticker Found. The message provides information about resistance at 275 for a potential trade idea but does not clearly indicate a buy or sell action for any specific ticker.
5551 Sell SPY

The message suggests closing out short positions on SPY as the per

Response before splitting: 5678 Neither SNOW. The message indicates the user is observing the stock without providing a clear indication of a buy or sell action.
5679 Sell VIX

The message suggests a sell action for VIX as it mentions that the price has rebounded but is being held near the low of the day, indicating weakness in the upward movement.
5680 Sell GOOG. The term "puts" indicates a bearish sentiment on the stock. The mention of "took a 10% trim on the first play" suggests the user may have already profited from a bearish position on GOOG.
5681 No Ticker Found
5682 Neither GS

This message does not clearly indicate whether to buy or sell GS. It simply mentions that there is more GS stock available.
5683 Neither GOOGL

The message does not provide a clear buy or sell signal for GOOGL. It encourages readers to conduct their own due diligence instead.
5684 Neither No Ticker Found

- The message does not provide a specific signal to buy or sell any specific stock. It simply sugges

invalid literal for int() with base 10: 'Reasoning:'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'The'
invalid literal for int() with base 10: 'Sell'
invalid literal for int() with base 10: 'Buy'
invalid literal for int() with base 10: 'Buy'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: '-'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'The'
invalid literal for int() with base 10: 'The'
invalid literal for int() with base 10: '-'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: 'Reasoning:'
invalid literal for int() with base 10: 'The'
invalid literal for int() with base 10: 'The'
invalid literal for int() with base 10: 'This'
invalid literal for int() with base 10: '-'
invalid literal f

In [134]:
display(stock_df)

,Username,Message,Timestamp,ChatGPT_Response,Stock,Buy or Sell
7,waysotheraccount,lcid up 20 percent for 7c 33 to 41,2023-07-12T15:18:10.090000+00:00,Sell LCID. The message indicates that LCID is ...,LCID,Sell
12,Waxui Alerts 🍭,NFLX Day Trade idea Closed poorly today after ...,2023-08-09T20:35:04.320000+00:00,Sell NFLX. The message indicates a sell signal...,NFLX,Sell
16,Waxui Alerts 🍭,Trim SPX here250 - 320 28%Holding most,2023-08-28T14:24:18.964000+00:00,Sell SPX,SPX,Sell
17,Waxui Alerts 🍭,Stopped out of NVDA here @150,2023-06-08T14:45:39.107000+00:00,"Sell NVDA. The phrase ""stopped out"" typically ...",NVDA,Sell
22,waysotheraccount,estc Had good er and uptrend in progress 75c a...,2023-09-01T15:22:25.527000+00:00,Buy ESTC,ESTC,Buy
...,...,...,...,...,...,...
5875,waysotheraccount,added some sava 20c for Jan 2024,2023-08-23T15:29:12.960000+00:00,Buy SAVA,SAVA,Buy
5877,Waxui Alerts 🍭,BOOM NVDA here150 - 225 50%Holding 1/2,2023-04-14T17:22:50.047000+00:00,Buy NVDA,NVDA,Buy
5879,waysotheraccount,tsat calls up over 100 percent from last week,2023-08-15T16:53:02.739000+00:00,Sell TSAT. The message indicates that the pric...,TSAT,Sell
5881,Waxui Alerts 🍭,Stopped out of DOCU The larger set up still lo...,2024-01-24T16:29:16.934000+00:00,Sell DOCU,DOCU,Sell
